# Imports

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset, Planetoid, LRGBDataset
from torch_geometric.transforms import NormalizeFeatures, Constant, OneHotDegree
from torch_geometric.nn import GCNConv, GINConv, GATv2Conv, GPSConv, global_mean_pool, Linear
import torch.nn.functional as F
from torch.nn.functional import cross_entropy
from torch_geometric.loader import DataLoader

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Cora: Node Classification

In [20]:
dataset = Planetoid(root='.', name='Cora', transform=NormalizeFeatures())
data = dataset[0]

train_dataset = data.train_mask
test_dataset = data.test_mask
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]

print()
print(data)
print('=============================================================')

# stats about the first graph
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')

dataset = dataset.shuffle()

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')


Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Has isolated nodes: False
Has self-loops: False
Number of training graphs: 2708
Number of test graphs: 2708


In [32]:
dataset = Planetoid(root='.', name='Cora', transform=NormalizeFeatures())
data = dataset[0]
train_mask = data.train_mask
test_mask = data.test_mask

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        return x.log_softmax(dim=-1)

class GIN(torch.nn.Module):
    def __init__(self):
        super(GIN, self).__init__()
        nn1 = torch.nn.Sequential(torch.nn.Linear(dataset.num_features, 16), torch.nn.ReLU(), torch.nn.Linear(16, 16))
        nn2 = torch.nn.Sequential(torch.nn.Linear(16, 16), torch.nn.ReLU(), torch.nn.Linear(16, 16))
        self.conv1 = GINConv(nn1)
        self.conv2 = GINConv(nn2)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        return x.log_softmax(dim=-1)


class GATv2(torch.nn.Module):
    def __init__(self):
        super(GATv2, self).__init__()
        self.conv1 = GATv2Conv(dataset.num_features, 16)
        self.conv2 = GATv2Conv(16, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        return x.log_softmax(dim=-1)

def train(model):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = torch.nn.functional.nll_loss(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

def test(model, mask):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=-1)
    correct = pred[mask].eq(data.y[mask]).sum().item()
    acc = correct / mask.sum().item()
    return acc

hidden_channels = [64, 64, 64]
models = [GCN(), GIN(), GATv2()]

for model in models:
    print(f"Training {model.__class__.__name__}")
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    for epoch in range(51):
        loss = train(model)
        train_acc = test(model, data.train_mask)
        test_acc = test(model, data.test_mask)
        if epoch % 10 == 0:
            print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}')

Training GCN
Epoch: 000, Train Acc: 0.4500, Test Accuracy: 0.2400
Epoch: 010, Train Acc: 0.8786, Test Accuracy: 0.5540
Epoch: 020, Train Acc: 0.9429, Test Accuracy: 0.6850
Epoch: 030, Train Acc: 0.9500, Test Accuracy: 0.7130
Epoch: 040, Train Acc: 0.9571, Test Accuracy: 0.7350
Epoch: 050, Train Acc: 0.9643, Test Accuracy: 0.7630
Training GIN
Epoch: 000, Train Acc: 0.1429, Test Accuracy: 0.1490
Epoch: 010, Train Acc: 0.2714, Test Accuracy: 0.2990
Epoch: 020, Train Acc: 0.5429, Test Accuracy: 0.4780
Epoch: 030, Train Acc: 0.8143, Test Accuracy: 0.5790
Epoch: 040, Train Acc: 0.9714, Test Accuracy: 0.6410
Epoch: 050, Train Acc: 0.9929, Test Accuracy: 0.6470
Training GATv2
Epoch: 000, Train Acc: 0.1500, Test Accuracy: 0.1330
Epoch: 010, Train Acc: 0.8143, Test Accuracy: 0.5600
Epoch: 020, Train Acc: 0.9214, Test Accuracy: 0.7570
Epoch: 030, Train Acc: 0.9500, Test Accuracy: 0.7490
Epoch: 040, Train Acc: 0.9571, Test Accuracy: 0.7550
Epoch: 050, Train Acc: 0.9500, Test Accuracy: 0.7370


In [26]:
class GPSConvNet(nn.Module):
    def __init__(self, hidden_channels, heads=1, dropout=0.0, act='relu'):
        super(GPSConvNet, self).__init__()
        self.GPSConvs = nn.ModuleList()
        prev_channels = dataset.num_node_features
        h = hidden_channels[0]
        self.preprocess = nn.Sequential(Linear(dataset.num_node_features, 2 * h), nn.GELU(), Linear(2 * h, h), nn.GELU())
        for h in hidden_channels:
            gatv2_conv = GATv2Conv(h, h // heads, heads=heads, dropout=dropout)
            gps_conv = GPSConv(h, gatv2_conv, heads=4, dropout=dropout, act=act)
            self.GPSConvs.append(gps_conv)
            prev_channels = h
        self.final_conv = GATv2Conv(prev_channels, dataset.num_classes, heads=1, dropout=dropout)
        
    def forward(self, x, edge_index):
        x = self.preprocess(x)
        for gps_conv in self.GPSConvs:
            x = x.float()
            x = F.relu(gps_conv(x, edge_index))
            x = F.dropout(x, p=0.6, training=self.training)
        x = self.final_conv(x, edge_index)
        return F.log_softmax(x, dim=1)

hidden_channels = [64, 64, 64]

model = GPSConvNet(hidden_channels)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = torch.nn.functional.nll_loss(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

def test(model, mask):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=-1)
    correct = pred[mask].eq(data.y[mask]).sum().item()
    acc = correct / mask.sum().item()
    return acc

for epoch in range(50):
    loss = train()
    train_acc = test(model, data.train_mask)
    test_acc = test(model, data.test_mask)
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}')



Epoch: 000, Train Acc: 0.2071, Test Accuracy: 0.1310
Epoch: 010, Train Acc: 0.2643, Test Accuracy: 0.1690
Epoch: 020, Train Acc: 0.4429, Test Accuracy: 0.2830
Epoch: 030, Train Acc: 0.8357, Test Accuracy: 0.5280
Epoch: 040, Train Acc: 0.9143, Test Accuracy: 0.6170


# Enzyme: Graph Classification

In [21]:
dataset = TUDataset(root='.', name='ENZYMES')
data = dataset[0]

train_dataset = dataset[:450]
test_dataset = dataset[450:]
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]

print()
print(data)
print('=============================================================')

print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')

dataset = dataset.shuffle()

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')


Dataset: ENZYMES(600):
Number of graphs: 600
Number of features: 3
Number of classes: 6

Data(edge_index=[2, 168], x=[37, 3], y=[1])
Number of nodes: 37
Number of edges: 168
Average node degree: 4.54
Has isolated nodes: False
Has self-loops: False
Number of training graphs: 450
Number of test graphs: 150


In [4]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

dataset = TUDataset(root='.', name='ENZYMES')
dataset = dataset.shuffle()

train_dataset = dataset[:450]
test_dataset = dataset[450:]

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = global_mean_pool(x, batch)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        return x
    
class GIN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GIN, self).__init__()
        self.conv1 = GINConv(Linear(dataset.num_node_features, hidden_channels))
        self.conv2 = GINConv(Linear(hidden_channels, hidden_channels))
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = global_mean_pool(x, batch)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)       
        return x

class GATv2(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super(GATv2, self).__init__()
        self.conv1 = GATv2Conv(dataset.num_node_features, hidden_channels, heads=heads)
        self.conv2 = GATv2Conv(hidden_channels * heads, hidden_channels, heads=heads)
        self.lin = Linear(hidden_channels * heads, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = global_mean_pool(x, batch)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x) 
        return x
        

gcn_model = GCN(hidden_channels=64)
gin_model = GIN(hidden_channels=64)
gatv2_model = GATv2(hidden_channels=64, heads=8)

models = [gcn_model, gin_model, gatv2_model]

def train(model):
    model.train()
    for data in train_loader:
         out = model(data.x, data.edge_index, data.batch) 
         loss = criterion(out, data.y)
         loss.backward()
         optimizer.step()
         optimizer.zero_grad()

def test(model, loader):
     model.eval()
     correct = 0
     for data in loader:
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)
         correct += int((pred == data.y).sum())
     return correct / len(loader.dataset)

for model in models:

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = torch.nn.CrossEntropyLoss()
    print(f"Training {model.__class__.__name__}")

    for epoch in range(1, 1001):
        train(model)
        train_acc = test(model, train_loader)
        test_acc = test(model, test_loader)
        if epoch % 100 == 0:
            print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Training GCN
Epoch: 100, Train Acc: 0.2244, Test Acc: 0.1867
Epoch: 200, Train Acc: 0.2667, Test Acc: 0.2333
Epoch: 300, Train Acc: 0.2822, Test Acc: 0.2867
Epoch: 400, Train Acc: 0.2778, Test Acc: 0.2867
Epoch: 500, Train Acc: 0.2822, Test Acc: 0.3067
Epoch: 600, Train Acc: 0.2756, Test Acc: 0.3067
Epoch: 700, Train Acc: 0.2844, Test Acc: 0.3067
Epoch: 800, Train Acc: 0.3222, Test Acc: 0.3200
Epoch: 900, Train Acc: 0.2956, Test Acc: 0.3000
Training GIN
Epoch: 100, Train Acc: 0.2844, Test Acc: 0.2800
Epoch: 200, Train Acc: 0.2600, Test Acc: 0.2867
Epoch: 300, Train Acc: 0.3111, Test Acc: 0.2400
Epoch: 400, Train Acc: 0.3044, Test Acc: 0.2733
Epoch: 500, Train Acc: 0.3467, Test Acc: 0.3200
Epoch: 600, Train Acc: 0.3422, Test Acc: 0.3200
Epoch: 700, Train Acc: 0.3400, Test Acc: 0.2933
Epoch: 800, Train Acc: 0.3333, Test Acc: 0.2867
Epoch: 900, Train Acc: 0.3533, Test Acc: 0.3600
Training GATv2
Epoch: 100, Train Acc: 0.2511, Test Acc: 0.2200
Epoch: 200, Train Acc: 0.2511, Test Acc: 0.2600

In [6]:
class GPSConvNet(nn.Module):
    def __init__(self, hidden_channels, heads=1, dropout=0.0, act='relu'):
        super(GPSConvNet, self).__init__()
        self.GPSConvs = nn.ModuleList()
        h = hidden_channels[0]
        self.preprocess = nn.Sequential(Linear(dataset.num_node_features, 2 * h), nn.GELU(), Linear(2 * h, h), nn.GELU())
        for h in hidden_channels:
            gatv2_conv = GATv2Conv(h, h // heads, heads=heads, dropout=dropout)
            gps_conv = GPSConv(h, gatv2_conv, heads=4, dropout=dropout, act=act)
            self.GPSConvs.append(gps_conv)  
        self.final_lin = Linear(hidden_channels[-1], dataset.num_classes)

    def forward(self, x, edge_index, batch):
        x = self.preprocess(x)
        for gps_conv in self.GPSConvs:
            x = x.float()
            x = F.relu(gps_conv(x, edge_index))
            x = F.dropout(x, p=0.6, training=self.training)
        x = global_mean_pool(x, batch)
        x = self.final_lin(x)
        return F.log_softmax(x, dim=1)

hidden_channels = [64, 64, 64]
dataset = TUDataset(root='.', name='ENZYMES')
data = dataset[0]

model = GPSConvNet(hidden_channels)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

train_dataset = dataset[:450]
test_dataset = dataset[450:]
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def train():
    model.train()

    for data in train_loader:
         out = model(data.x, data.edge_index, data.batch)
         loss = criterion(out, data.y)
         loss.backward()
         optimizer.step()
         optimizer.zero_grad()

def test(loader):
     model.eval()

     correct = 0
     for data in loader:
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)
         correct += int((pred == data.y).sum())
     return correct / len(loader.dataset)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
print(f"Training {model.__class__.__name__}")
# 
for epoch in range(1, 51):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Training GPSConvNet
Epoch: 010, Train Acc: 0.3067, Test Acc: 0.0000
Epoch: 020, Train Acc: 0.3000, Test Acc: 0.0000
Epoch: 030, Train Acc: 0.3156, Test Acc: 0.0000
Epoch: 040, Train Acc: 0.3200, Test Acc: 0.0000
Epoch: 050, Train Acc: 0.3311, Test Acc: 0.0867


# IMDB: Graph Classification

In [22]:
dataset = TUDataset(root='.', name='IMDB-BINARY')

data = dataset[0]

train_dataset = dataset[:850]
test_dataset = dataset[850:]
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]

print()
print(data)
print('=============================================================')

print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')

dataset = dataset.shuffle()

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')


Dataset: IMDB-BINARY(1000):
Number of graphs: 1000
Number of features: 0
Number of classes: 2

Data(edge_index=[2, 146], y=[1], num_nodes=20)
Number of nodes: 20
Number of edges: 146
Average node degree: 7.30
Has isolated nodes: False
Has self-loops: False
Number of training graphs: 850
Number of test graphs: 150


In [17]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

dataset = TUDataset(root='.', name='IMDB-BINARY', transform=Constant())
dataset = dataset.shuffle()

train_dataset = dataset[:850]
test_dataset = dataset[850:]

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = global_mean_pool(x, batch)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        return x
    
class GIN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GIN, self).__init__()
        self.conv1 = GINConv(Linear(dataset.num_node_features, hidden_channels))
        self.conv2 = GINConv(Linear(hidden_channels, hidden_channels))
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = global_mean_pool(x, batch)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        return x

class GATv2(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super(GATv2, self).__init__()
        self.conv1 = GATv2Conv(dataset.num_node_features, hidden_channels, heads=heads)
        self.conv2 = GATv2Conv(hidden_channels * heads, hidden_channels, heads=heads)
        self.lin = Linear(hidden_channels * heads, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = global_mean_pool(x, batch)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)    
        return x
        

gcn_model = GCN(hidden_channels=64)
gin_model = GIN(hidden_channels=64)
gatv2_model = GATv2(hidden_channels=64, heads=8)

models = [gcn_model, gin_model, gatv2_model]

def train(model):
    model.train()

    for data in train_loader:
         out = model(data.x, data.edge_index, data.batch)
         loss = criterion(out, data.y)
         loss.backward()
         optimizer.step()
         optimizer.zero_grad()

def test(model, loader):
     model.eval()

     correct = 0
     for data in loader:
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)
         correct += int((pred == data.y).sum())
     return correct / len(loader.dataset)

for model in models:

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = torch.nn.CrossEntropyLoss()
    print(f"Training {model.__class__.__name__}")

    for epoch in range(1, 1001):
        train(model)
        train_acc = test(model, train_loader)
        test_acc = test(model, test_loader)
        if epoch % 100 == 0:
            print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Training GCN
Epoch: 100, Train Acc: 0.5012, Test Acc: 0.4933
Epoch: 200, Train Acc: 0.5012, Test Acc: 0.4933
Epoch: 300, Train Acc: 0.5012, Test Acc: 0.4933
Epoch: 400, Train Acc: 0.6224, Test Acc: 0.5867
Epoch: 500, Train Acc: 0.6259, Test Acc: 0.5600
Epoch: 600, Train Acc: 0.6059, Test Acc: 0.5400
Epoch: 700, Train Acc: 0.6071, Test Acc: 0.5467
Epoch: 800, Train Acc: 0.6059, Test Acc: 0.5400
Epoch: 900, Train Acc: 0.6259, Test Acc: 0.5533
Epoch: 1000, Train Acc: 0.6071, Test Acc: 0.5600
Training GIN
Epoch: 100, Train Acc: 0.6329, Test Acc: 0.6400
Epoch: 200, Train Acc: 0.6471, Test Acc: 0.6267
Epoch: 300, Train Acc: 0.6565, Test Acc: 0.6333
Epoch: 400, Train Acc: 0.6341, Test Acc: 0.6133
Epoch: 500, Train Acc: 0.6376, Test Acc: 0.6133
Epoch: 600, Train Acc: 0.5882, Test Acc: 0.5667
Epoch: 700, Train Acc: 0.6365, Test Acc: 0.6133
Epoch: 800, Train Acc: 0.6553, Test Acc: 0.6267
Epoch: 900, Train Acc: 0.6682, Test Acc: 0.6400
Epoch: 1000, Train Acc: 0.6094, Test Acc: 0.5867
Training GAT

In [51]:
class GPSConvNet(nn.Module):
    def __init__(self, hidden_channels, heads=1, dropout=0.0, act='relu'):
        super(GPSConvNet, self).__init__()
        self.GPSConvs = nn.ModuleList()
        h = hidden_channels[0]
        self.preprocess = nn.Sequential(Linear(dataset.num_node_features, 2 * h), nn.GELU(), Linear(2 * h, h), nn.GELU())
        for h in hidden_channels:
            gatv2_conv = GATv2Conv(h, h // heads, heads=heads, dropout=dropout)
            gps_conv = GPSConv(h, gatv2_conv, heads=4, dropout=dropout, act=act)
            self.GPSConvs.append(gps_conv)  
        self.final_lin = Linear(hidden_channels[-1], dataset.num_classes)

    def forward(self, x, edge_index, batch):
        x = self.preprocess(x)
        for gps_conv in self.GPSConvs:
            x = x.float()
            x = F.relu(gps_conv(x, edge_index))
            x = F.dropout(x, p=0.6, training=self.training)
        x = global_mean_pool(x, batch)
        x = self.final_lin(x)
        return F.log_softmax(x, dim=1)

hidden_channels = [64, 64, 64]
dataset = TUDataset(root='.', name='IMDB-BINARY', transform=OneHotDegree(max_degree=140))
data = dataset[0]

model = GPSConvNet(hidden_channels)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

train_dataset = dataset[:850]
test_dataset = dataset[850:]
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def train():
    model.train()

    for data in train_loader:
         out = model(data.x, data.edge_index, data.batch)
         loss = criterion(out, data.y)
         loss.backward()
         optimizer.step()
         optimizer.zero_grad()

def test(loader):
     model.eval()

     correct = 0
     for data in loader:
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)
         correct += int((pred == data.y).sum())
     return correct / len(loader.dataset)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
print(f"Training {model.__class__.__name__}")

for epoch in range(1, 21):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Training GPSConvNet
Epoch: 010, Train Acc: 0.7588, Test Acc: 0.5667
Epoch: 020, Train Acc: 0.7682, Test Acc: 0.7333


# PascalVOC-SP: Node Classification

In [23]:
dataset = LRGBDataset(root='.', name='PascalVOC-SP')
data = dataset[0]

train_dataset = dataset[:7000]
test_dataset = dataset[7000:]
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]

print()
print(data)
print('=============================================================')

print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')

dataset = dataset.shuffle()

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')


Dataset: LRGBDataset(8498):
Number of graphs: 8498
Number of features: 14
Number of classes: 21

Data(x=[460, 14], edge_index=[2, 2632], edge_attr=[2632, 2], y=[460])
Number of nodes: 460
Number of edges: 2632
Average node degree: 5.72
Has isolated nodes: False
Has self-loops: False
Number of training graphs: 6798
Number of test graphs: 1700


In [23]:
torch.manual_seed(12345)
dataset = dataset.shuffle()

train_dataset = dataset[:7000]
test_dataset = dataset[7000:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Number of training graphs: 7000
Number of test graphs: 1498


In [5]:
# for step, data in enumerate(train_loader):
#     print(f'Step {step + 1}:')
#     print('=======')
#     print(f'Number of graphs in the current batch: {data.num_graphs}')
#     print(data)
#     print()

In [2]:
dataset = LRGBDataset(root='.', name='PascalVOC-SP')

train_dataset = dataset[:7000]
test_dataset = dataset[7000:]

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)
    
class GIN(torch.nn.Module):
    def __init__(self):
        super(GIN, self).__init__()
        nn1 = torch.nn.Sequential(torch.nn.Linear(dataset.num_node_features, 16), torch.nn.ReLU(), torch.nn.Linear(16, 16))
        self.conv1 = GINConv(nn1)
        self.bn1 = torch.nn.BatchNorm1d(16)
        nn2 = torch.nn.Sequential(torch.nn.Linear(16, dataset.num_classes))
        self.conv2 = GINConv(nn2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, data.edge_index)
        return F.log_softmax(x, dim=1)    

class GATv2(torch.nn.Module):
    def __init__(self):
        super(GATv2, self).__init__()
        self.conv1 = GATv2Conv(dataset.num_features, 8, heads=8)
        self.conv2 = GATv2Conv(8 * 8, dataset.num_classes, heads=1)

    def forward(self, data):
        x = self.conv1(data.x, data.edge_index)
        x = F.relu(x)
        x = self.conv2(x, data.edge_index)
        return F.log_softmax(x, dim=1)

def train(model, optimizer, train_loader):
    model.train()
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()

def test(model, test_loader):
    model.eval()
    test_correct = 0
    total_nodes = 0
    for data in test_loader:
        with torch.no_grad():
            out = model(data)
            pred = out.argmax(dim=-1)
            test_correct += pred.eq(data.y).sum().item()
            total_nodes += data.num_nodes
    test_acc = test_correct / total_nodes
    return test_acc


models = [GCN(), GIN(), GATv2()]

for model in models:
    print(f"Training {model.__class__.__name__}")
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    for epoch in range(3):
        train(model, optimizer, train_loader)
        train_acc = test(model, train_loader)
        test_acc = test(model, test_loader)
        if epoch % 2 == 0:
            print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')



Training GCN
Epoch: 000, Train Acc: 0.6995, Test Acc: 0.6950
Epoch: 002, Train Acc: 0.6995, Test Acc: 0.6950
Training GIN
Epoch: 000, Train Acc: 0.6995, Test Acc: 0.6950
Epoch: 002, Train Acc: 0.6995, Test Acc: 0.6950
Training GATv2
Epoch: 000, Train Acc: 0.6935, Test Acc: 0.6890
Epoch: 002, Train Acc: 0.6656, Test Acc: 0.6602


In [54]:
class GPSConvNet(nn.Module):
    def __init__(self, hidden_channels, heads=1, dropout=0.0, act='relu'):
        super(GPSConvNet, self).__init__()
        self.GPSConvs = nn.ModuleList()
        h = hidden_channels[0]
        self.preprocess = nn.Sequential(Linear(dataset.num_node_features, 2 * h), nn.GELU(), Linear(2 * h, h), nn.GELU())
        for h in hidden_channels:
            gatv2_conv = GATv2Conv(h, h // heads, heads=heads, dropout=dropout)
            gps_conv = GPSConv(h, gatv2_conv, heads=4, dropout=dropout, act=act)
            self.GPSConvs.append(gps_conv)  
        self.final_lin = Linear(hidden_channels[-1], dataset.num_classes)

    def forward(self, x, edge_index, batch):
        x = self.preprocess(x)
        for gps_conv in self.GPSConvs:
            x = x.float()
            x = F.relu(gps_conv(x, edge_index))
            x = F.dropout(x, p=0.6, training=self.training)
        x = global_mean_pool(x, batch)
        x = self.final_lin(x)
        return F.log_softmax(x, dim=1)

hidden_channels = [64, 64, 64]
dataset = LRGBDataset(root='.', name='PascalVOC-SP')
data = dataset[0]

model = GPSConvNet(hidden_channels)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

train_dataset = dataset[:7000]
test_dataset = dataset[7000:]
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def train():
    model.train()

    for data in train_loader:
         out = model(data.x, data.edge_index, data.batch)
         loss = criterion(out, data.y)
         loss.backward()
         optimizer.step()
         optimizer.zero_grad()

def test(loader):
     model.eval()

     correct = 0
     for data in loader:
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)
         correct += int((pred == data.y).sum())
     return correct / len(loader.dataset)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()
print(f"Training {model.__class__.__name__}")

for epoch in range(1, 21):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Training GPSConvNet
